In [108]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [109]:
# state_govt=soup.find('section',id='Central')

In [110]:
# head=state_govt.find_next('tr')
# header=head.find_all('th')
# header_list=[header[i].text.split("\r\n")[1].strip() for i in range(len(header))]
# header_list=['Sr No',
#  'Subsidy scheme',
#  'Scheme Type',
#  'Ministry',
#  'Description',
#  'Visiting link to know more',
#  'Applicability Central/State']

# dic={}
# for j in header_list:
#     dic[j]=[]

In [111]:
def get_subsidy_data(ID,dic):
    govt=soup.find('section',id=ID)
    head=govt.find_next('tr')
    header=head.find_all('th')

    pointer=head.find_next('tr')
    while pointer:
        try:
            row=pointer.find_all("td")
            dic["Sr No"].append(row[0].text.strip())
            dic["Subsidy scheme"].append(row[1].text.strip())
            dic["Scheme Type"].append(row[2].text.strip())
            dic["Ministry"].append(row[3].text.strip())
            dic["Description"].append(row[4:len(row)-2][0].text.strip())
            dic['Visiting link to know more'].append(row[-2].text.strip())
            dic['Applicability Central/State'].append(row[-1].text.strip())

            skip_rows_count=0
            tab=pointer.find_next('table')
            for _ in range(len(pointer.find_all('table'))):
                skip_rows_count+=len(tab.find_all('tr'))
                tab=tab.find_next('table')
            # script to extract table in csv format

            # while table:
            #     # print(table)
            #     headers = [th.get_text(strip=True) for th in table.find_all('th')]
            #     rows = []
            #     for tr in table.find_all('tr')[1:]:
            #         cells = tr.find_all('td')
            #         row = [cell.get_text(separator='\n', strip=True) for cell in cells]
            #         rows.append(row)
            #     print(rows)
            #     df=pd.DataFrame(rows,columns=headers)
            #     df.to_csv("temp.csv",index=False)
            #     print(df)
            #     table=table.find_next('table')

            for _ in range(skip_rows_count+1):
                pointer=pointer.find_next('tr')
        except Exception:
            break
    

In [112]:
categories=["Women","All","SCST"]

header_list=['Sr No',
 'Subsidy scheme',
 'Scheme Type',
 'Ministry',
 'Description',
 'Visiting link to know more',
 'Applicability Central/State']

dic={}
for j in header_list:
    dic[j]=[]

for cat in categories:
    url = f'https://www.standupmitra.in/Home/SubsidySchemesFor{cat}'
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    for ID in ["Central","State"]:
        get_subsidy_data(ID=ID,dic=dic)

c:\Users\gupta\Desktop\policies\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.standupmitra.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\gupta\Desktop\policies\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.standupmitra.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\gupta\Desktop\policies\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.standupmitra.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [113]:
df=pd.DataFrame(dic)
df.to_csv("subsidies.csv",index=False)

In [ ]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="llama3") 

def split_description(subs):
    message=f"""
    You are provided with a description of a government subsidy scheme : {subs}\
    Return a list with two elements, first one as the summary of the scheme, and second as the eligibility criteria\
    Give a concise eligibility cirteria.
    """

    response=model.invoke(message).split("**")
    summary=response[2].strip(" \n*")
    criteria=response[4].strip(" \n*")

    return summary,criteria

In [ ]:
import pandas as pd

df=pd.read_csv("subsidies.csv")

summary=[]
criteria=[]

for _,subsidy in df.iterrows():
    response=split_description(subsidy["Description"])
    summary.append(response[0])
    criteria.append(response[1])

df["summary"]=summary
df["eligibility_criteria"]=criteria

df.to_csv("processed_subsidies.csv",index=False)